In [ ]:
import os
import pandas as pd
import soccerdata as sd
import logging
from sqlalchemy import create_engine
from dotenv import load_dotenv
import warnings

warnings.filterwarnings('ignore')

pd.options.mode.chained_assignment = None

load_dotenv()

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)



In [ ]:
db_url = (
    f"postgresql+psycopg2://{os.getenv('PGUSER')}:{os.getenv('PASSWORD')}"
    f"@{os.getenv('HOST')}:{os.getenv('PORT')}/{os.getenv('DB')}"
)
engine = create_engine(db_url)

In [ ]:
# Seasons to fetch
seasons = list(range(2005, 2010))

all_team_match_stats = pd.DataFrame()

for season in seasons:
    logging.info(f"Fetching team match stats for season: {season}")
    
    fb = sd.FBref(leagues="Big 5 European Leagues Combined", seasons=[season])
    
    try:
        season_stats = fb.read_team_match_stats(stat_type='schedule', opponent_stats=False)
        logging.info(f"Fetched {len(season_stats)} rows for season {season}")
        
        all_team_match_stats = pd.concat([all_team_match_stats, season_stats], ignore_index=True)
        
    except Exception as e:
        logging.error(f"Failed to fetch data for season {season}: {e}")

logging.info(f"Total rows fetched across all seasons: {len(all_team_match_stats)}")



In [ ]:

logging.info("Writing combined team match stats to table")

batch_size = 25  # small batch to avoid parameter limits
for start in range(0, len(all_team_match_stats), batch_size):
    batch = all_team_match_stats.iloc[start:start+batch_size]
    
    batch.to_sql(
        'FBref_team_match_stats',
        con=engine,
        schema='bronze',
        if_exists='replace',
        index=False,
        method='multi'  # sends multiple rows per INSERT, avoids parameter limits
    )
    


In [ ]:
all_team_match_stats.head(15)